In [0]:
!pip install tensorflow-gpu=='2.1.0'

     |████████████████████████████████| 421.8MB 44kB/s 
     |████████████████████████████████| 450kB 51.7MB/s 
     |████████████████████████████████| 3.8MB 59.6MB/s 
     |████████████████████████████████| 81kB 13.4MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goo

In [0]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Lambda, MaxPool2D, Dropout, Conv2DTranspose, concatenate, Input
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import math
import zipfile
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# getting permission to get files from Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download and unzip images and masks
downloaded = drive.CreateFile({'id':"1NU-skxQaoE7CphcS_KfsuiqSXJG1sC1U"})
downloaded.GetContentFile("img.zip")
with zipfile.ZipFile("img.zip", 'r') as zip_ref:
    zip_ref.extractall("img")

downloaded = drive.CreateFile({'id':"1O1JeORPoGmVdRf8G_L9MME__rbp4xBph"})
downloaded.GetContentFile("masks.zip")
with zipfile.ZipFile("masks.zip", 'r') as zip_ref:
    zip_ref.extractall("masks")

downloaded = drive.CreateFile({'id':"149nbJpCK6Ezlg7HMxEIT-vJOwV-9iD0U"})
downloaded.GetContentFile("masks_val.zip")
with zipfile.ZipFile("masks_val.zip", 'r') as zip_ref:
    zip_ref.extractall("masks_val")

downloaded = drive.CreateFile({'id':"1XZDIzBk_NqZk4ONK_rsL8mFFCyv_3BUX"})
downloaded.GetContentFile("img_val.zip")
with zipfile.ZipFile("img_val.zip", 'r') as zip_ref:
    zip_ref.extractall("img_val")

downloaded = drive.CreateFile({'id':"18We85QdAPGV6k8v2uILIme8ReTah6Kad"})
downloaded.GetContentFile("tiramisu_103_layers.json")

In [0]:
num_images = len(os.listdir("img/img")) #number of training images
img_size = 224
x_train = np.array([[[[]]]])
y_train = np.array([[[[]]]])
# we read every image, resize it and add it to a numpy array.
for i in os.listdir("img/img"):
    img = cv2.imread("img/img/" + i)
    x_train = np.append(x_train, cv2.resize(img, (img_size, img_size)))

# masks are also read and resized
for i in os.listdir("masks/masks"):
    img = cv2.imread("masks/masks/" + i, 0)
    y_train = np.append(y_train, cv2.resize(img, (img_size, img_size)))

# the masks have grayscale value 0 or 2. We need 0 or 1
for i in range(len(y_train)):
    if y_train[i] >= 1:
        y_train[i] = 1

# reshaping after appending data
x_train = np.reshape(x_train, (num_images, img_size, img_size, 3))
y_train = np.reshape(y_train, (num_images, img_size, img_size))

In [0]:
num_images = len(os.listdir("img_val/img_val"))
img_size = 224
x_val = np.array([[[[]]]])
y_val = np.array([[[[]]]])
for i in os.listdir("img_val/img_val"):
    img = cv2.imread("img_val/img_val/" + i)
    x_val = np.append(x_val, cv2.resize(img, (img_size, img_size)))

for i in os.listdir("masks_val/masks_val"):
    img = cv2.imread("masks_val/masks_val/" + i, 0)
    y_val = np.append(y_val, cv2.resize(img, (img_size, img_size)))

for i in range(len(y_val)):
    if y_val[i] >= 1:
        y_val[i] = 1

x_val = np.reshape(x_val, (num_images, img_size, img_size, 3))
y_val = np.reshape(y_val, (num_images, img_size, img_size))

In [0]:
# U-net architecture
inputs = Input((img_size, img_size, 3))
s = Lambda(lambda x: x / 255) (inputs) #normalise values between 0 and 1

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPool2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPool2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPool2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPool2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 224, 224, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_114 (Conv2D)             (None, 224, 224, 16) 448         lambda_6[0][0]                   
__________________________________________________________________________________________________
dropout_54 (Dropout)            (None, 224, 224, 16) 0           conv2d_114[0][0]                 
____________________________________________________________________________________________

In [0]:
# training the model
model.fit(x=x_train, y=y_train, validation_data=(x_val, y_val), epochs=30, batch_size=4)

In [0]:
im_number = 45 # which img to use for testing
plt.imshow(x_val[im_number]) #using plt to show the original image
plt.show()

# adding dimension as model requires batched data
img = np.expand_dims(x_val[im_number], axis=0)

# getting model predictions and removing axes for displaying
preds = model.predict(img)
preds = np.reshape(preds, (img_size, img_size))

# model returns float values. We are chainging them to 0 or 1
for i in range(len(preds)):
    for j in range(len(preds[i])):
        if preds[i][j] >= 0.5:
            preds[i][j] = 255
        else:
            preds[i][j] = 0

plt.imshow(preds)
plt.show()